### Application of Divide and Conquer in multiplying polynomials

- We can apply divide and conquer on polynomial multiplcation to get more efficient product

- Suppose $A(x) = 3x^2 + 2x + 5, B(x) = 5x^2 + x + 2$. Then $A(x) B(x) = 15x^4 + 13x^3 + 33 x^2 + 9x + 10$

- General problems statement
    - **Input:** 2 $n-1$ degree polynomials $A = a_{n-1} x^{n-1} + a_{n-2} x^{n-2} + ... a_1 x + a_0$ and $B = b_{n-1} x^{n-1} + b_{n-2} x^{n-2} + ... b_1 x + b_0$
    - **Output:** The product polynomial $C = c_{2n-2} x^{2n-2} + c_{2n-3} x^{2n-3} + ... c_1 x + c_0$. Note the following relation between $a$, $b$ and $c$ coefficients:
        - $c_{2n-2} = a_{n-1} \cdot b_{n-1}$
        - $c_{2n-3} = a_{n-1} \cdot b_{n-2} + a_{n-2} \cdot b_{n-1}$ 
        - $c_{2n-4} = a_{n-1} \cdot b_{n-3} + a_{n-2} \cdot b_{n-2} + a_{n-3} \cdot b_{n-1}$ 
        - $...$
        - $c_2 = a_{2} \cdot b_{0} + a_{1} \cdot b_{1} + a_{0} \cdot b_{2}$
        - $c_1 = a_{1} \cdot b_{0} + a_{0} \cdot b_{1}$
        - $c_0 = a_{0} \cdot b_{0}$

- Rewriting the previous example
    - **Input:** n=3, A = (3,2,5), B=(5,1,2)
    - **Output:** C = (15, 13, 33, 9, 10)     

- Naive solution
    - To solve this, we can iterate through the coefficients in arrays $A$ and $B$. For every pair, we compute the value, and add it into the appropriate slot for $c$
    - This is $O(N^2)$, so it's obviously bad

In [1]:
def naive_mult_poly(A, B, n):
    '''
    time complexity: O(N^2) because of nested for loop
    space complexity: O(N)
    '''
    product = [0] * ((2*n)-1)
    for i in range(n-1):
        for j in range(n-1):
            product[i+j] += product[i+j] + (A[i] * B[j])
    return product


- Naive divide and conquer
    - Surprisingly, there is a way to apply divide and conquer to solve this problem, though it isn't straightforward. 
    - Let's first try a naive divide and conquer approach
    
    - Let the 2 polynomials be $A = a_{n-1} x^{n-1} + a_{n-2}x^{n-2} + ... + a_{1}x + a_0$ and $B = b_{n-1}x^{n-1} + b_{n-2}x^{n-2} + ... + b_{1}x + b_0$. 

- To get the recursion, notice that we can rewrite $A = C \cdot x^{\frac{n}{2}} + D$ and $B = E \cdot x^{\frac{n}{2}} + F$ 
    $$A = (a_{n-1} x^{\frac{n}{2} - 1} + a_{n-2} x^{\frac{n}{2} - 2} + ... + a_{n - \frac{n}{2}} x^{\frac{n}{2} - \frac{n}{2}}) \cdot x^{\frac{n}{2}} + (a_{\frac{n}{2} - 1} x^{\frac{n}{2} - 1} + a_{\frac{n}{2} - 2} x^{\frac{n}{2} - 2} + a_{\frac{n}{2} - \frac{n}{2}} x^{\frac{n}{2} - \frac{n}{2}})$$
    - Basically we factor out $x^{\frac{n}{2}}$ from all polynomial terms with $x^a \ge x^{\frac{n}{2}}$
    - Notice that after factorising, the coefficient of $x^{\frac{n}{2}}$ is just another polynomial. We can apply the same logic, and split that into a sum of its own sub polynomials!
    - This iterative halving of the coefficient is the basis of the recursion

- What is the base case of the recursion?
    - This splitting of higher order polynomials will go on until we hit this case; $A = a_1 x + a_0$ and $B = b_1 x + b_0$, where the coefficients are no longer polynomials
    - Multiplying it out, it is obvious that $A \cdot B = a_1 b_1 x^2 + (a_1 b_0 + a_0 b_1) \cdot x + a_0 b_0$
    - So in the base case, we perform 4 operations; $(a_1 b_1, a_1 b_0, a_0 b_1, a_0 b_0)$

- So clearly, at every step, 1 operation becomes 4! At each step, the amount of work is:
    - Step 0: $4^0$
    - Step 1: $4^1$
    - Step 2: $4^2$
    - ...
    - Step n: $4^n$

- How many times can we split the polynomial of degree $n$? 
    - Obviously, since we are halving the power at each step, we can do this for $\log_2(n)$ steps

- Recurrence relation: $T(n) = 4 T(\frac{n}{2}) + kn$
    - 4 because we are doing 4 operations
    - $n/2$ because we are halving the polynomial degree
    - $kn$ because we are taking the sum of $n$ coefficients, multiplied by some constant work $k$

- So the total work done is $\sum_{i=0}^{\log_2(n)} 4^i$, 
    - This is $O(4^{\log_2(n)}) = O(2^{2 \log_2(n)}) = O(2^{\log_2(n^2)}) = O(n^2)$


In [103]:
def naive_dnc_mult_poly(A, B, input_length, log_recurse_level=0):
    '''
    time complexity: O(N^2), from analysis above
    '''
    # log_recurse_level+=1
    # print('='*50)
    # print(f'Calling {A=} and {B=} and {input_length=}')
    # print(f"{log_recurse_level=}")

    if len(A) == len(B) == 1:
        ret = [A[0] * B[0]]
        # print(f"Returning {ret=}")
        return ret
    
    if input_length % 2 != 0:
        input_length += 1

    if (len(A) < input_length):
        A = A + (input_length-len(A)) * [0] 
    if (len(B) < input_length):
        B = (input_length-len(B)) * [0] + B
    
    assert(len(A) == len(B))
    
    
    # print(f'Modified {A=} and {B=} and {input_length=}')
    mid_a = len(A) // 2
    mid_b = len(B) // 2

    left_half_array = naive_dnc_mult_poly(A[:mid_a], B[:mid_b], mid_a, log_recurse_level)
    right_half_array = naive_dnc_mult_poly(A[mid_a:], B[mid_b:], mid_a, log_recurse_level)

    c1 = naive_dnc_mult_poly(A[:mid_a], B[mid_b:], mid_a, log_recurse_level)
    c2 = naive_dnc_mult_poly(A[mid_a:], B[:mid_b], mid_a, log_recurse_level)
    mid_array = [v1+v2 for v1,v2 in zip(c1, c2)]

    # print(f"Pre-length mod: {left_half_array=}, {mid_array=}, {right_half_array=}")
    
    expected_length = input_length*2 - 1
    add_to_left_half_end = expected_length - len(left_half_array)
    add_to_right_half_start = expected_length - len(right_half_array)
    add_to_mid_end = (expected_length - len(mid_array)) // 2
    add_to_mid_start = (expected_length - len(mid_array)) - add_to_mid_end
    # print(f"{expected_length=}, {add_to_left_half_end=}, {add_to_right_half_start=}, {add_to_mid_start=}, {add_to_mid_end=}")

    left_half_array = left_half_array + [0]*add_to_left_half_end
    right_half_array = [0]*add_to_right_half_start + right_half_array
    mid_array = [0]*add_to_mid_start + mid_array + [0]*add_to_mid_end
    
    final = [x+y+z for x,y,z in zip(left_half_array, mid_array, right_half_array)]
    # print(f"Combining: {left_half_array=}, {mid_array=}, {right_half_array=}")
    # print(f"{final=}")
    return final

A = [4,3,2,1]
B = [0,3,2,1]
naive_dnc_mult_poly(A, B, len(A), log_recurse_level=0)

[0, 12, 17, 16, 10, 4, 1]

- So we have taken our original naive solution which was $O(N^2)$, and applied divide and conquer to get...another $O(N^2)$ solution. Not amazing

- But we can be smarter about how we do this! 
    - Recall that we're rewriting polynomials $A$ and $B$ as $A = C x^{\frac{n}{2}} + D$ and $B = E x^{\frac{n}{2}} + F$ 
    - As we discussed above, when multiplying out $A$ and $B$, we're doing 4 multiplications $a_1 b_1, a_1 b_2, a_2 b_1, a_2 b_2$

 